
## Overview

This notebook shows you how to create and query a table or DataFrame loaded from data stored in AWS S3. There are two ways to establish access to S3: [IAM roles](https://docs.databricks.com/user-guide/cloud-configurations/aws/iam-roles.html) and access keys.

*We recommend using IAM roles to specify which cluster can access which buckets. Keys can show up in logs and table metadata and are therefore fundamentally insecure.* If you do use keys, you'll have to escape the `/` in your keys with `%2F`.

This is a **Python** notebook so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` magic command. Python, Scala, SQL, and R are all supported.

In [0]:
## Importing required libraries
from pyspark.sql.functions import col

In [0]:
# File location and type
file_location = "/FileStore/tables/sample_csv-3.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

id,name,age
50,James,35
51,Harry,25
52,Garnet,30


In [0]:
# Checking the data type
df.printSchema()

root
 |-- id: integer (nullable = true)
 |--  name: string (nullable = true)
 |--  age: integer (nullable = true)



In [0]:
# Create a view or table / converting Dataframe into SQL table

df.createOrReplaceTempView("sample")

In [0]:
%sql
/* Query the created temp table in a SQL cell */
select * from sample

id,name,age
50,James,35
51,Harry,25
52,Garnet,30


Method 1: Renaming colum using withColumnRenamed

In [0]:
df1 = df.withColumnRenamed("id", "id_new").withColumnRenamed(" name", "name_new").withColumnRenamed(" age", "age_new")
df1.show()

+------+--------+-------+
|ID_New|Name_New|Age_New|
+------+--------+-------+
|    50|   James|     35|
|    51|   Harry|     25|
|    52|  Garnet|     30|
+------+--------+-------+



Method 2: Renaming colum using selectExpr

In [0]:
df2 = df.selectExpr("id_new as ID_New", "name_new as Name_New", "age_new as Age_New")
df2.show()

+------+--------+-------+
|ID_New|Name_New|Age_New|
+------+--------+-------+
|    50|   James|     35|
|    51|   Harry|     25|
|    52|  Garnet|     30|
+------+--------+-------+



Method 3: Renaming colum using select

In [0]:
df.show()

+------+--------+-------+
|ID_New|Name_New|Age_New|
+------+--------+-------+
|    50|   James|     35|
|    51|   Harry|     25|
|    52|  Garnet|     30|
+------+--------+-------+



In [0]:
df3 = df.select(
    col("ID_New").alias("ID_Newest"),
    col("Name_New").alias("Name_Newest"),
    col("Age_New").alias("Age_Newest")
)
df3.show()

+---------+-----------+----------+
|ID_Newest|Name_Newest|Age_Newest|
+---------+-----------+----------+
|       50|      James|        35|
|       51|      Harry|        25|
|       52|     Garnet|        30|
+---------+-----------+----------+

